In [ ]:
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import datetime
import numpy as np
import os
import csv

from sklearn.preprocessing import MinMaxScaler
from keras import layers, losses, Sequential
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [ ]:
scaler = MinMaxScaler()
benign = np.loadtxt('dataset/1.benign.csv', delimiter=",", skiprows=1)
X_train = benign[:40000]
X_test0 = benign[40000:]
X_test1 = np.loadtxt("dataset/1.mirai.scan.csv", delimiter=",", skiprows=1)
X_test2 = np.loadtxt("dataset/1.mirai.ack.csv", delimiter=",", skiprows=1)
X_test3 = np.loadtxt("dataset/1.mirai.syn.csv", delimiter=",", skiprows=1)
X_test4 = np.loadtxt("dataset/1.mirai.udp.csv", delimiter=",", skiprows=1)
X_test5 = np.loadtxt("dataset/1.mirai.udpplain.csv", delimiter=",", skiprows=1)

In [ ]:
class Autoencoder(Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Sequential([
            layers.Dense(115, activation="relu"),
            layers.Dense(86, activation="relu"),
            layers.Dense(57, activation="relu"),
            layers.Dense(37, activation="relu"),
            layers.Dense(28, activation="relu")
        ])
        self.decoder = Sequential([
            layers.Dense(37, activation="relu"),
            layers.Dense(57, activation="relu"),
            layers.Dense(86, activation="relu"),
            layers.Dense(115, activation="sigmoid")
        ])
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
x = scaler.fit_transform(X_train)
            
# create an autoencoder model
ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['accuracy'])
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor,tensorboard_callback]
)

In [ ]:
%tensorboard --logdir logs/fit